<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/gpu_nv_topk_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -h

In [ ]:
!nvidia-smi -q -d SUPPORTED_CLOCKS

In [ ]:
!nvidia-smi -i 0 -ac 1215,1410

In [ ]:
!git clone https://github.com/ZhangJingrong/gpu_topK_benchmark.git

[**Parallel Top-K Algorithms on GPU: A Comprehensive Study and New Methods**](https://sc23.supercomputing.org/presentation/?id=pap294&sess=sess156) [paper](https://dl.acm.org/doi/pdf/10.1145/3581784.3607062)

# REPRODUCIBILITY OF EXPERIMENTS

Our experiments are conducted with NVIDIA CUDA 12.0 on
NVIDIA A100 GPU.
To reproduce the results in our paper:

1) Set up the Python environment for figure generation: Go to
directory “script”, and run “bash setup.sh”.

2) Download and compile required libraries: Run the “download.sh” in the directory “thrid_party”. Follow the description in
README.md to compile these libraries.

3) Generate the binary for the benchmark: In the directory
“benchmark”, run “make”. A binary file named “benchmark” is generated.

4) Run the experiments and collect data for the table and figures:
In the directory “script”, run “bash run-k.sh” to collect data for Fig.6
and “bash run-n.sh” to collect data for Fig.7.

5) Use Python code to generate the table and figures: In the
directory “script”, run “bash exp.sh”.
Running “bash run-k.sh” and “bash run-n.sh” takes about 8.38
hours and 8.65 hours, respectively.
After running “bash run-k.sh”, we can get a file named “k-asx.out.a100”, which contains the running time for batch size 1, N
equaling 2^15,2^20,2^25 and 2^30, and K varying from 2^3 to 2^20.
After running “bash run-n.sh”, we can acquire a file named “n-asx.out.a100”, which contains the running time for batch sizes 1 and
100, K equaling 2^5, 2^8, and 2^15, and N varying from 2^11 to
2^30.
With “exp.sh”, we can get two figures and a CSV file in the
directory “script”: “k-as-x-a100.png” corresponds to Fig.6; “n-as-xa100.png” corresponds to Fig.7; “speedup.csv” corresponds to Table
2

In [ ]:
!cd gpu_topK_benchmark/third_party && bash download.sh

## build

In [6]:
!cd gpu_topK_benchmark/third_party/faiss && make


g++ -std=c++14 -O2 -fPIC -DNDEBUG -I. -I/usr/local/cuda/include  -c -o faiss/gpu/GpuResources.o faiss/gpu/GpuResources.cpp
/usr/local/cuda/bin/nvcc -std=c++14 -O2 -Xcompiler -fPIC -arch=sm_80 -DNDEBUG -I. -I/usr/local/cuda/include -c faiss/gpu/utils/DeviceUtils.cu -o faiss/gpu/utils/DeviceUtils.o
/usr/local/cuda/bin/nvcc -std=c++14 -O2 -Xcompiler -fPIC -arch=sm_80 -DNDEBUG -I. -I/usr/local/cuda/include -c faiss/gpu/utils/BlockSelectFloat.cu -o faiss/gpu/utils/BlockSelectFloat.o
/usr/local/cuda/bin/nvcc -std=c++14 -O2 -Xcompiler -fPIC -arch=sm_80 -DNDEBUG -I. -I/usr/local/cuda/include -c faiss/gpu/utils/WarpSelectFloat.cu -o faiss/gpu/utils/WarpSelectFloat.o
/usr/local/cuda/bin/nvcc -std=c++14 -O2 -Xcompiler -fPIC -arch=sm_80 -DNDEBUG -I. -I/usr/local/cuda/include -c faiss/gpu/utils/blockselect/BlockSelectFloat128.cu -o faiss/gpu/utils/blockselect/BlockSelectFloat128.o
/usr/local/cuda/bin/nvcc -std=c++14 -O2 -Xcompiler -fPIC -arch=sm_80 -DNDEBUG -I. -I/usr/local/cuda/include -c faiss/gp

In [8]:
!cd gpu_topK_benchmark/third_party/gpu_selection && mkdir build && cd build && cmake .. && make


-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 11.8.89
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Configuring done (2.3s)
CMake Warning (dev) in lib/CMakeLists.txt:
  Policy CMP0104 is not set: CMAKE_CUDA_ARCHITECTURES now detected for NVCC,
  empty CUDA_ARCHITECTURES not allowed.  Run "cmake --help-policy CMP0104"
  for policy details.  Use the cmake_policy command to set the policy and
  suppress this warning.

  CUDA_ARCHITECTURES is empty for target "gpu_selection".
This warning is for project developers.  Use -Wno-dev to suppress it.

CMake War

In [9]:
!cd gpu_topK_benchmark/third_party/gpu_selection && mv build/lib/libgpu_selection.so ./


In [21]:
!ls gpu_topK_benchmark/third_party/faiss/
!ls gpu_topK_benchmark/third_party/gpu_selection/
!ls gpu_topK_benchmark/third_party/

faiss  libfaiss.so  Makefile
app    CMakeLists.txt  lib		    LICENSE	    LICENSE-cub
build  include	       libgpu_selection.so  LICENSE-catch2  README
download.sh  faiss	    libgridselect.so  radix.patch  raft.patch  spdlog
DrTopKSC     gpu_selection  LICENSE.faiss     raft	   rmm


In [39]:
!make -C gpu_topK_benchmark/benchmark

make: Entering directory '/content/gpu_topK_benchmark/benchmark'
nvcc -c -O2 -std=c++17 -Xcompiler "-Wall -Wextra -Wno-unused-parameter" --expt-relaxed-constexpr --extended-lambda -arch=sm_80 -DNDEBUG -I../include -I../third_party/faiss -isystem ../third_party/DrTopKSC/bitonic/LargerKVersions/largerK/ -I../third_party/DrTopKSC/baseline+filter+beta+shuffle/ -I../third_party/gpu_selection/include -I../third_party/gpu_selection/lib -isystem ../third_party/raft/cpp/include -isystem ../third_party/rmm/include -I../third_party/spdlog/include -I../third_party -o benchmark.o benchmark.cu
nvcc -arch=sm_80 -L../third_party/faiss -lfaiss -Xlinker -rpath=../third_party/faiss -L../third_party/gpu_selection -lgpu_selection -Xlinker -rpath=../third_party/gpu_selection/ -L../third_party -lgridselect -Xlinker -rpath=../third_party -lcurand -o benchmark benchmark.o
make: Leaving directory '/content/gpu_topK_benchmark/benchmark'


In [24]:
#!ldd gpu_topK_benchmark/benchmark/benchmark
!cd gpu_topK_benchmark/benchmark && ldd benchmark

	linux-vdso.so.1 (0x00007ffe495e3000)
	libfaiss.so => ../third_party/faiss/libfaiss.so (0x00007c928f530000)
	libgpu_selection.so => ../third_party/gpu_selection/libgpu_selection.so (0x00007c928cf40000)
	libgridselect.so => ../third_party/libgridselect.so (0x00007c928c554000)
	libcurand.so.10 => /usr/local/cuda/targets/x86_64-linux/lib/libcurand.so.10 (0x00007c9285c00000)
	libstdc++.so.6 => /lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007c92859d4000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007c92858ed000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007c928c51c000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007c92856c5000)
	/lib64/ld-linux-x86-64.so.2 (0x00007c92908ba000)
	libcublas.so.11 => /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11 (0x00007c927fa00000)
	librt.so.1 => /lib/x86_64-linux-gnu/librt.so.1 (0x00007c928c515000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007c928c510000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.s

## benchmark

In [25]:
!cd gpu_topK_benchmark/benchmark && ./benchmark

usage: ./benchmark [-c] [-n niter] [-w warmup_niter] [-r num_random_bits] [-g] algos batch_size len k
   -c: compare with cpu result for validation
   -n: number of iterations for benchmark, default = 1
   -w: number of iterations for warmup, default = 1
   -r: number of random bits, can only be used for 32-bits type and can't be used with -g
   -g: use Gaussian distribution data (mean=0, stddev=1.0)
       meaningful only for float
       with this option, len should be multiple of 2 for float

default distribution for test data:
       Uniform distribution in (0,1] for float
       bit-wise randomness for other data types

algos: comma separated algorithm names, e.g. cpu,cub
available algorithms:
    cpu
    cub
    drtopk_bitonic
    drtopk_radix
    faiss_block
    faiss_warp
    grid_select
    raft_radix_11bits_extra_pass
    sampleselect
    sampleselect-bucket
    sampleselect-quick


In [46]:
!cd gpu_topK_benchmark/benchmark && ./benchmark -c -w 20 -n 100 cpu 1 1e6 20

     2.847, 

In [40]:
!sleep 3600

^C


In [ ]:
!cd gpu_topK_benchmark/script && bash run-k.sh

In [ ]:
!cd gpu_topK_benchmark/script && bash run-n.sh

## setup python env for benchmark exp



In [ ]:
!cd gpu_topK_benchmark/script && bash setup.sh

In [ ]:
!cd gpu_topK_benchmark/script && bash exp.sh